# Deep Learning model to detect suspicious DNS TXT records 

This notebook uses a pre-trained deep learning model to predict whether a DNS TXT record is suspicious or not.

While DNS TXT records initially were supposed to only hold descriptive text, its use and format have been subject to changes and debates over the years. This makes TXT records ideal candidates for malicious and unintended practices which include adding large records to create more efficient DNS amplification attacks, or creating a command and control channel for malware. 

The model is trained independently and is then made available for download. The DNS TXT records are categorized into commonly identified types like email, verification, http using regular expressions https://www.tide-project.nl/blog/wtmc2020/. The TXT records that do not match regular expressions for well known types are labeled as 1 for "unknown/suspicious" and otherwise 0 for "not suspicious". The deep learning model we have developed uses DNX TXT responses to analyze patters of character sequences to predict of a DNS TXT is suspicious or not. Higher the pred_is_unknown_proba, the more likely is the DNS TXT record is suspicious. The threshold for flagging a domain as suspicious is set at 0.5.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import numpy as np 
import pandas as pd 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from torch.autograd import Variable
from torch.optim import lr_scheduler
from collections import Counter,OrderedDict
import pickle

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/detect_suspicious_dns_txt_records_using_pretrained_model_in_dsdl/"
vocab_size = 10002
embedding_dim = 64
hidden_size = 64
fc_hidden_size = 64
num_output_nodes = 1

dropout = 0.5

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("torch version: " + torch.__version__)

numpy version: 1.19.2
pandas version: 1.1.3
torch version: 1.11.0


## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DLTK Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DLTK app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter.

### Option 1 - pull data from Splunk

In [3]:
import libs.SplunkSearch as SplunkSearch

In [4]:
search = SplunkSearch.SplunkSearch()

In [5]:
df = search.as_df()
df

,_time,feature_0,feature_1,feature_2,i,s
0,2022-05-18T10:30:54.000+02:00,0.8460328145883977,0.32027013367041945,0.8921294794417918,1,1
1,2022-05-18T10:30:54.000+02:00,0.3228773274458945,0.9270845879800618,0.6180154159665108,2,2
2,2022-05-18T10:30:54.000+02:00,0.9652930609881878,0.6127289566211402,0.6959671885706484,3,0
3,2022-05-18T10:30:54.000+02:00,0.9985754322260618,0.7212827517651021,0.6265083532780409,4,1
4,2022-05-18T10:30:54.000+02:00,0.4984334879554808,0.4883861029520631,0.8144497619941831,5,2
5,2022-05-18T10:30:54.000+02:00,0.9335020575672388,0.7346397545188665,0.03047862323001027,6,0
6,2022-05-18T10:30:54.000+02:00,0.8374560391530395,0.9159072837792337,0.01808864250779152,7,1
7,2022-05-18T10:30:54.000+02:00,0.8355274042114615,0.3842416317202151,0.1918810959905386,8,2
8,2022-05-18T10:30:54.000+02:00,0.35425164457410574,0.11845849128440022,0.01499031251296401,9,0
9,2022-05-18T10:30:54.000+02:00,0.38959994819015265,0.7069222498685122,0.930438622366637,10,1


### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

| makeresults count=10<br>
| streamstats c as i<br>
| fit MLTKContainer mode=stage algo=barebone_template i into app:barebone_template

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("barebone_template")

In [ ]:
df

In [ ]:
param

## Stage 2 - create and initialize a model

In [3]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
v = CountVectorizer(analyzer='char_wb', ngram_range=(2, 4), lowercase=True)
analyzer = v.build_analyzer()
def tokenize(s):
    tokens = analyzer(s)
    
    try:
        tokens.remove(' ')
    except ValueError:
        pass
    tokens = list(map(str.strip, tokens))
    return tokens

class DataFrameDataset(Dataset):

    def __init__(self, df):
        self.texts = []
        self.labels = []
        for i, row in df.iterrows():
            label = row.is_unknown
            text = row.text
            self.texts.append(text)
            self.labels.append(label)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx],self.labels[idx]


class DNSTxtClassifier(nn.Module):
        def __init__(self, vocab_size, embed_dim, hidden_size, fc_hidden_size,dropout=0.5,layers=1):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size,embed_dim,padding_idx=0)
            self.lstm = nn.LSTM(embed_dim, hidden_size,batch_first=True,dropout=dropout, num_layers=1)
            self.fc2 = nn.Linear(hidden_size, 1)
            self.sigmoid = nn.Sigmoid()
#lengths.to(device)
        def forward(self, text, lengths):
            out = self.embedding(text)
            out = nn.utils.rnn.pack_padded_sequence(out,lengths.detach().numpy() , enforce_sorted=False, batch_first=True )
            out, (hidden, cell) = self.lstm(out)
            out = hidden[-1, :, :]
            out = self.fc2(out)
            out = self.sigmoid(out)
            return out
        


def init(df,param):
    model = DNSTxtClassifier(vocab_size, embedding_dim, hidden_size,fc_hidden_size,dropout = dropout)
    model.load_state_dict(torch.load(MODEL_DIRECTORY+'/detect_suspicious_dns_txt_records_using_pretrained_model_in_dsdl.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    model.eval()
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
# apply your model
# returns the calculated results
vocab = pickle.load(open(MODEL_DIRECTORY+"vocab.pkl", 'rb'))

def remove_spaces_key_value(text):
     return text.replace(' = ','=').replace(" =","=").replace('= ','=').replace(' - ','-').replace(" -","-").replace('- ','-').replace(' : ',":").replace(': ' ,":").replace(' :',':')

def get_token_index(text):
    idxs = []
    tokens = tokenize(text)
    for token in tokens:
        if token not in vocab:
            idxs.append(vocab['<unk>'])
        else:
            idxs.append(vocab[token])
    return idxs

def preprocess_text(text):
    text = remove_spaces_key_value(text)
    text = text.replace('"','').replace("'","").lower()
    return text


def apply(model,df,param):
    is_unknown_probability_score = []
    is_unknown = []
    for idx, row in df.iterrows():
        text = row['text']
        text = preprocess_text(text)
        indexed_text = get_token_index(text)
        length = [len(indexed_text)]
        token_tensor = torch.LongTensor(indexed_text).to(device)             
        token_tensor = token_tensor.unsqueeze(1).T                            
        length_tensor = torch.LongTensor(length) 
        pred_proba = model(token_tensor, length_tensor)[:, 0].item()
        pred  = int(pred_proba >= 0.5)
        is_unknown_probability_score.append(pred_proba)
        is_unknown.append(pred)
    output = pd.DataFrame()
    output['predicted_is_unknown'] = is_unknown_probability_score
    output['pred_is_unknown'] = is_unknown
    return output

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = DNSTxtClassifier(vocab_size, embedding_dim, hidden_size,fc_hidden_size,dropout = dropout)
    model.load_state_dict(torch.load(MODEL_DIRECTORY+'/detect_suspicious_dns_txt_records_using_pretrained_model_in_dsdl.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    model.eval()
    return model 

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DLTK app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [ ]:
import libs.SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [ ]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

In [ ]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [ ]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code